In [ ]:
# This block is to install ase, tk for the GUI, and ipywidgets-jsonschema
import sys

print(sys.executable)
# !{sys.executable} -m pip install ase
# !{sys.executable} -m pip install tk
import tkinter

print(tkinter.Tcl().eval("info patchlevel"))
# !{sys.executable} -m pip install ipywidgets-jsonschema


/Users/mat3ra/.pyenv/versions/3.11.4/bin/python
8.6.13


/Users/mat3ra/.pyenv/versions/3.11.4/bin/python


KeyboardInterrupt: 

In [ ]:
# data that comes from JS, here are some examples
poscars = {
    "Ni": """Ni - primitive
1.0
   2.130422000	   0.000000000	   1.230000000
   0.710140800	   2.008582000	   1.230000000
   0.000000000	   0.000000000	   2.460000000
Ni
1
Selective dynamics
direct
   0.000000000    0.000000000    0.000000000 T T T Ni
""",
    "Cu": """Cu4
1.0
   3.5774306715697510    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.5774306715697510    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.5774306715697510
Cu
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Cu
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Cu
""",
    "Au": """Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
""",
    "SiC": """Si4 C4
1.0
   4.3539932475828609    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.3539932475828609    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.3539932475828609
Si C
4 4
direct
   0.7500000000000000    0.2500000000000000    0.7500000000000000 Si4+
   0.7500000000000000    0.7500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.2500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.7500000000000000    0.7500000000000000 Si4+
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C4-
   0.0000000000000000    0.5000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.0000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.5000000000000000    0.0000000000000000 C4-
""",
    "Graphene": """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
""",
}


In [128]:
import io
from ase.build import surface, make_supercell
from ase.io import read, write
from ase import Atoms
import numpy as np
from ase.visualize import view

# Define a function to read POSCAR content and return an Atoms object
def poscar_to_atoms(poscar_content):
    poscar_io = io.StringIO(poscar_content)
    atoms = read(poscar_io, format="vasp")
    return atoms

# Define a function to create a surface and supercell
def prepare_surface_and_supercell(atoms, miller_indices, layers, vacuum, superlattice_matrix):
    # Create the surface
    surface_atoms = surface(atoms, miller_indices, layers, vacuum)
    # Create the supercell
    supercell_atoms = make_supercell(surface_atoms, superlattice_matrix)
    return supercell_atoms

# Define a function to calculate the scaling factors for lattice vectors
def calculate_scaling_factors(substrate_cell, layer_cell):
    # Calculate the norm of the a and b vectors for both substrate and layer
    substrate_a_norm = np.linalg.norm(substrate_cell[0])
    substrate_b_norm = np.linalg.norm(substrate_cell[1])
    layer_a_norm = np.linalg.norm(layer_cell[0])
    layer_b_norm = np.linalg.norm(layer_cell[1])
    
    # Calculate the scaling factors for a and b
    scale_a = substrate_a_norm / layer_a_norm
    scale_b = substrate_b_norm / layer_b_norm
    
    return scale_a, scale_b

# Define a function to apply scaling to the layer's lattice vectors
def scale_layer(layer_atoms, scale_a, scale_b):
    # Scale the a and b vectors by the calculated factors
    layer_cell = layer_atoms.get_cell()
    layer_cell[0] *= scale_a
    layer_cell[1] *= scale_b
    layer_atoms.set_cell(layer_cell, scale_atoms=True)
    
    return layer_atoms

# Define a function to calculate strain on the layer
def calculate_strain(original_layer_cell, scaled_layer_cell):
    # Calculate the original and scaled norms
    original_a_norm = np.linalg.norm(original_layer_cell[0])
    original_b_norm = np.linalg.norm(original_layer_cell[1])
    scaled_a_norm = np.linalg.norm(scaled_layer_cell[0])
    scaled_b_norm = np.linalg.norm(scaled_layer_cell[1])
    
    # Calculate the strain along a and b
    strain_a = (scaled_a_norm - original_a_norm) / original_a_norm
    strain_b = (scaled_b_norm - original_b_norm) / original_b_norm
    
    return strain_a, strain_b

# Define the main function to create the interface
def create_interface(substrate_poscar, layer_poscar, surface_params, superlattice_matrices, z_offset):
    # Convert POSCAR content to Atoms objects
    substrate_atoms = poscar_to_atoms(substrate_poscar)
    layer_atoms = poscar_to_atoms(layer_poscar)
    
    # Prepare the substrate surface and supercell
    substrate_supercell = prepare_surface_and_supercell(
        substrate_atoms, 
        (surface_params["miller"]["h"], surface_params["miller"]["k"], surface_params["miller"]["l"]),
        surface_params["number_of_layers"], 
        surface_params["vacuum"], 
        superlattice_matrices["slab"]
    )
    
    # Prepare the layer supercell
    layer_supercell = prepare_surface_and_supercell(
        layer_atoms, 
        (0, 0, 1),  # Assuming the layer is already a 2D material, just a single layer
        1,  # Only one layer needed for a 2D material
        0,  # No additional vacuum needed, it's already in the POSCAR
        superlattice_matrices["layer"]
    )

    # Calculate scaling factors
    scale_a, scale_b = calculate_scaling_factors(substrate_supercell.get_cell(), layer_supercell.get_cell())
    
    # Save the original layer cell for strain calculations
    original_layer_cell = layer_supercell.get_cell().copy()
    
    # Scale the layer
    scaled_layer_supercell = scale_layer(layer_supercell, scale_a, scale_b)
    
    # Calculate strain on the layer
    strain_a, strain_b = calculate_strain(original_layer_cell, scaled_layer_supercell.get_cell())
    print("Strain along a:", strain_a)
    print("Strain along b:", strain_b)


    # Adjust Z position based on the Z offset
    z_max_substrate = max(substrate_supercell.positions[:, 2])
    layer_supercell.positions[:, 2] += z_max_substrate + z_offset
    
    # Combine substrate and layer into one Atoms object
    interface = substrate_supercell + layer_supercell
    
    return interface

# Define a function to compute the total strain
def compute_total_strain(substrate_cell, layer_cell):
    scale_a, scale_b = calculate_scaling_factors(substrate_cell, layer_cell)
    scaled_layer_cell = layer_cell.copy()
    scaled_layer_cell[0] *= scale_a
    scaled_layer_cell[1] *= scale_b
    strain_a, strain_b = calculate_strain(layer_cell, scaled_layer_cell)
    total_strain = np.sqrt(strain_a**2 + strain_b**2)
    return strain_a, strain_b, total_strain

from ase.geometry import get_distances

def original_nearest_neighbor_distance(atoms):
    # Calculate distances between all pairs of atoms
    distances = get_distances(atoms.positions, cell=atoms.cell, pbc=atoms.pbc)[1]
    # The nearest neighbor distance is the smallest non-zero distance
    nn_distance = np.min(distances[distances > 0])
    return nn_distance

def is_layer_distorted(layer_atoms, original_nn_distance, max_increase_percentage):
    new_nn_distance = original_nearest_neighbor_distance(layer_atoms)
    return new_nn_distance > original_nn_distance * (1 + max_increase_percentage / 100)

# Brute force search for the best superlattice matrices
def search_best_matrices(substrate_atoms, layer_atoms, min_val, max_val, max_increase_percentage):
    results = []
    original_layer_nn_distance = original_nearest_neighbor_distance(layer_atoms)
    
    for i_sub in range(min_val, max_val+1):
        for j_sub in range(min_val, max_val+1):
            for k_sub in range(min_val, max_val+1):
                for l_sub in range(min_val, max_val+1):
                    slab_matrix = np.array([[i_sub, j_sub, 0], [k_sub, l_sub, 0], [0, 0, 1]])
                    if np.linalg.det(slab_matrix) <= 0:
                        continue  # Skip singular matrix

                    for i_layer in range(min_val, max_val+1):
                        for j_layer in range(min_val, max_val+1):
                            for k_layer in range(min_val, max_val+1):
                                for l_layer in range(min_val, max_val+1):
                                    layer_matrix = np.array([[i_layer, j_layer, 0], [k_layer, l_layer, 0], [0, 0, 1]])
                                    if np.linalg.det(layer_matrix) <= 0:
                                        continue  # Skip singular matrix

                                    # Create supercells
                                    layer_supercell = make_supercell(layer_atoms, layer_matrix)
                                    if is_layer_distorted(layer_supercell, original_layer_nn_distance, max_increase_percentage):
                                        continue  # Skip if layer is too distorted

                                    # Calculate strains
                                    strain_a, strain_b, total_strain = compute_total_strain(
                                        substrate_atoms.get_cell(), layer_supercell.get_cell()
                                    )

                                    results.append({
                                        "strain_a": strain_a,
                                        "strain_b": strain_b,
                                        "total_strain": total_strain,
                                        "slab_matrix": slab_matrix.tolist(),
                                        "layer_matrix": layer_matrix.tolist()
                                    })

    # Sort results by total strain and return
    sorted_results = sorted(results, key=lambda x: x['total_strain'])
    return sorted_results


In [114]:

# Example usage
substrate_poscar = poscars["Au"]
layer_poscar = poscars["Graphene"]
surface_params = {
    "miller": {"h": 1, "k": 1, "l": 1},
    "number_of_layers": 3,
    "vacuum": 10,
}
superlattice_matrices = {
    "slab": [[1, 0, 0], [0, 1, 0], [0, 0, 1]],  # Example superlattice matrix for substrate
    "layer": [[2, 0 , 0], [0, 2, 0], [0, 0, 1]],  # Example superlattice matrix for layer
}
z_offset = 3.0  # Example Z offset

# Create the interface
interface_atoms = create_interface(substrate_poscar, layer_poscar, surface_params, superlattice_matrices, z_offset)

# Optional: Save the interface to a file
# write("interface_POSCAR.vasp", interface_atoms, format="vasp")

# Visualize the interface
view(interface_atoms*(3,3,1))
# print(interface_atoms)



Strain along a: 0.19545947636150526
Strain along b: 0.1954594651175555


In [129]:
substrate_poscar = poscars["Au"]
layer_poscar = poscars["Graphene"]
surface_params = {
    "miller": {"h": 1, "k": 1, "l": 1},
    "number_of_layers": 3,
    "vacuum": 10,
}

# Brute force seatrch for the best superlattice matrices
substrate_atoms = poscar_to_atoms(substrate_poscar)  # Use the actual POSCAR content
layer_atoms = poscar_to_atoms(layer_poscar)     
min_val, max_val = 0, 2  # Define the search range for the matrix elements

max_increase_percentage = 100

best_matrices = search_best_matrices(substrate_atoms, layer_atoms, min_val, max_val, max_increase_percentage)

# print all total strains
for result in best_matrices:
    print(result["total_strain"], result["slab_matrix"], result["layer_matrix"])

# Print the best result
if best_matrices:
    best_result = best_matrices[0]
    print("Best superlattice matrices with the smallest total strain:")
    print("Strain along a:", best_result["strain_a"])
    print("Strain along b:", best_result["strain_b"])
    print("Total strain:", best_result["total_strain"])
    print("Substrate superlattice matrix:", best_result["slab_matrix"])
    print("Layer superlattice matrix:", best_result["layer_matrix"])
else:
    print("No valid superlattice matrices found within the given range.")

# Visualize the best result
if best_matrices:
    superlattice_matrices = {
        "slab": best_result["slab_matrix"],
        "layer": best_result["layer_matrix"],
    }
    z_offset = 3.0  # Example Z offset
    
    interface_atoms = create_interface(substrate_poscar, layer_poscar, surface_params, superlattice_matrices, z_offset)
    view(interface_atoms*(3,3,1))
    print(interface_atoms)


0.03381605872036796 [[1, 0, 0], [0, 1, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 0, 0], [0, 2, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 0, 0], [1, 1, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 0, 0], [1, 2, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 0, 0], [2, 1, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 0, 0], [2, 2, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 1, 0], [0, 1, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 1, 0], [0, 2, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 1, 0], [1, 2, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 2, 0], [0, 1, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[1, 2, 0], [0, 2, 0], [0, 0, 1]] [[2, 1, 0], [1, 2, 0], [0, 0, 1]]
0.03381605872036796 [[2, 0, 0], 

TypeError: initial_value must be str or None, not Atoms